In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling1D,Conv1D
from keras.constraints import maxnorm

C:\Users\Home\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [10]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [12]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [13]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\Home\Anaconda\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [16]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [17]:
n_classes

6

- Defining the Architecture of LSTM

In [17]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [19]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 27s 4ms/step - loss: 1.3093 - acc: 0.4391 - val_loss: 1.1381 - val_acc: 0.4768
Epoch 2/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.9732 - acc: 0.5876 - val_loss: 0.9129 - val_acc: 0.6033
Epoch 3/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.7906 - acc: 0.6436 - val_loss: 0.8107 - val_acc: 0.5986
Epoch 4/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.7050 - acc: 0.6549 - val_loss: 0.9490 - val_acc: 0.5724
Epoch 5/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.6609 - acc: 0.6696 - val_loss: 0.7249 - val_acc: 0.6210
Epoch 6/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.6153 - acc: 0.6772 - val_loss: 0.7109 - val_acc: 0.6328
Epoch 7/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.5776 - acc: 0.7111 - val_loss: 0.6593 - val_acc: 0.7034
Epoch 8

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      413        73        0                   0   
STANDING                 0      107       425        0                   0   
WALKING                  0        0         0      465                  22   
WALKING_DOWNSTAIRS       0        0         0        1                 412   
WALKING_UPSTAIRS         0        0         0       32                   5   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            5  
STANDING                           0  
WALKING                            9  
WALKING_DOWNSTAIRS                 7  
WALKING_UPSTAIRS                 434  


In [21]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 363us/step


In [22]:
#  score

[0.42977901058258233, 0.9022734984730234]

In [43]:
Y_train.shape

(7352, 6)

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

## Basic CNN Model

In [84]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(Conv1D(32, 10, padding='same',input_shape=(timesteps, input_dim),kernel_initializer='glorot_normal',activation='relu'))
model.add(MaxPooling1D(5))
# Adding a dropout layer
model.add(Dropout(0.5))

model.add(Conv1D(32, 10, padding='same',input_shape=(timesteps, input_dim),kernel_initializer='glorot_normal',activation='relu'))
model.add(MaxPooling1D(5))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(Flatten())
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 128, 32)           2912      
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 25, 32)            0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 25, 32)            0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 25, 32)            10272     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 5, 32)             0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 5, 32)             0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 160)               0         
__________

In [85]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [86]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=50)

Train on 7352 samples, validate on 2947 samples
Epoch 1/50
7352/7352 [==============================] - 5s 613us/step - loss: 0.7439 - acc: 0.6854 - val_loss: 0.4671 - val_acc: 0.8616
Epoch 2/50
7352/7352 [==============================] - 4s 575us/step - loss: 0.2353 - acc: 0.9112 - val_loss: 0.3316 - val_acc: 0.8968
Epoch 3/50
7352/7352 [==============================] - 5s 631us/step - loss: 0.1632 - acc: 0.9373 - val_loss: 0.3357 - val_acc: 0.8918
Epoch 4/50
7352/7352 [==============================] - 4s 579us/step - loss: 0.1436 - acc: 0.9437 - val_loss: 0.3322 - val_acc: 0.9070
Epoch 5/50
7352/7352 [==============================] - 4s 593us/step - loss: 0.1273 - acc: 0.9465 - val_loss: 0.3102 - val_acc: 0.9084
Epoch 6/50
7352/7352 [==============================] - 4s 608us/step - loss: 0.1264 - acc: 0.9474 - val_loss: 0.3328 - val_acc: 0.9067
Epoch 7/50
7352/7352 [==============================] - 4s 586us/step - loss: 0.1206 - acc: 0.9501 - val_loss: 0.3462 - val_acc: 0.9094


In [81]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING        ...         WALKING_UPSTAIRS
True                              ...                         
LAYING                 510        ...                        2
SITTING                  0        ...                       19
STANDING                 0        ...                        1
WALKING                  0        ...                        1
WALKING_DOWNSTAIRS       0        ...                        3
WALKING_UPSTAIRS         1        ...                      453

[6 rows x 6 columns]


In [82]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 0s 154us/step


In [83]:
score

[0.30769265981690036, 0.9172039362063115]

## Hyper-parameter Tuning on different Parameters

In [18]:
def create_model(activation='relu',dropout_rate=0.0,init_mode='uniform',optimizer='adam',filters=32,kernel_size=5,pool_size=5):
    
    
    model = Sequential()
    model.add(Conv1D(filters, kernel_size, padding='same',input_shape=(128,9),kernel_initializer=init_mode,activation='relu'))
    model.add(MaxPooling1D(pool_size))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(filters, kernel_size, padding='same',input_shape=(128,9),kernel_initializer=init_mode, activation='relu'))
    model.add(MaxPooling1D(pool_size))
    model.add(Flatten())
    model.add(Dense(6, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [20]:

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model,verbose=2)
# define the grid search parameters
# n = [32,64,128]
optimizer = ['RMSprop', 'Adadelta', 'Adam']
init_mode = ['glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
dropout_rate = [0.5, 0.6, 0.7, 0.8]
batch_size = [16,32]
epochs = [30]
filters = [32,64,128]
kernel_size = [4,5,6,7]
pool_size = [5,6,7,8]
# learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]

param_grid = dict(optimizer=optimizer,init_mode=init_mode,dropout_rate=dropout_rate,batch_size=batch_size,epochs=epochs,
                  filters = filters,kernel_size=kernel_size,pool_size=pool_size)

rcv = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1,cv=6)
result = rcv.fit(X_train, Y_train)

# summarize results
print("Best: %f using %s" % (result.best_score_, result.best_params_))
means = result.cv_results_['mean_test_score']
stds = result.cv_results_['std_test_score']
params = result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\Home\Anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/30
 - 5s - loss: 0.2604 - acc: 0.8897
Epoch 2/30
 - 5s - loss: 0.1461 - acc: 0.9374
Epoch 3/30
 - 5s - loss: 0.0935 - acc: 0.9621
Epoch 4/30
 - 5s - loss: 0.0684 - acc: 0.9724
Epoch 5/30
 - 5s - loss: 0.0605 - acc: 0.9754
Epoch 6/30
 - 5s - loss: 0.0531 - acc: 0.9788
Epoch 7/30
 - 5s - loss: 0.0521 - acc: 0.9790
Epoch 8/30
 - 5s - loss: 0.0473 - acc: 0.9804
Epoch 9/30
 - 5s - loss: 0.0473 - acc: 0.9809
Epoch 10/30
 - 5s - loss: 0.0454 - acc: 0.9813
Epoch 11/30
 - 5s - loss: 0.0420 - acc: 0.9826
Epoch 12/30
 - 5s - loss: 0.0426 - acc: 0.9822
Epoch 13/30
 - 5s - loss: 0.0406 - acc: 0.9828
Epoch 14/30
 - 5s - loss: 0.0399 - acc: 0.9845
Epoch 15/30
 - 5s - loss: 0.0381 - acc: 0.9835
Epoch 16/30
 - 5s - loss: 0.0388 - acc: 0.9843
Epoch 17/30
 - 5s - loss: 0.0378 - acc: 0.9843
Epoch 18/30
 - 5s - loss: 0.0376 - acc: 0.9843
Epoch 19/30
 - 5s - loss: 0.0393 - acc: 0.9834
Epoch 20/30
 - 5s - loss: 0.0385 - acc: 0.9842
Epoch 21/30
 - 5s - loss: 0.0366 - acc: 0.9849
Epoch 22/30
 - 5s - lo

In [21]:
result.best_params_

{'pool_size': 5,
 'optimizer': 'Adadelta',
 'kernel_size': 6,
 'init_mode': 'he_normal',
 'filters': 64,
 'epochs': 30,
 'dropout_rate': 0.7,
 'batch_size': 32}

In [23]:
model = Sequential()
model.add(Conv1D(64, 6, padding='same',input_shape=(128,9),kernel_initializer='he_normal',activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.7))
model.add(Conv1D(64, 6, padding='same',input_shape=(128,9),kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
    # Compile model
model.compile(loss='binary_crossentropy', optimizer='Adadelta', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=30, batch_size=32,validation_data=(X_test, Y_test))

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 6s 836us/step - loss: 0.2621 - acc: 0.8893 - val_loss: 0.2537 - val_acc: 0.8927
Epoch 2/30
7352/7352 [==============================] - 6s 814us/step - loss: 0.1445 - acc: 0.9384 - val_loss: 0.2365 - val_acc: 0.9071
Epoch 3/30
7352/7352 [==============================] - 6s 825us/step - loss: 0.0907 - acc: 0.9640 - val_loss: 0.1683 - val_acc: 0.9334
Epoch 4/30
7352/7352 [==============================] - 6s 797us/step - loss: 0.0705 - acc: 0.9725 - val_loss: 0.1429 - val_acc: 0.9430
Epoch 5/30
7352/7352 [==============================] - 6s 790us/step - loss: 0.0590 - acc: 0.9768 - val_loss: 0.1267 - val_acc: 0.9477
Epoch 6/30
7352/7352 [==============================] - 6s 823us/step - loss: 0.0542 - acc: 0.9776 - val_loss: 0.1072 - val_acc: 0.9589
Epoch 7/30
7352/7352 [==============================] - 6s 796us/step - loss: 0.0516 - acc: 0.9784 - val_loss: 0.1100 - val_acc: 0.9557


In [24]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      433        33        0                   0   
STANDING                 0      118       410        3                   0   
WALKING                  0        0         0      485                  10   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0        1                  16   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           25  
STANDING                           1  
WALKING                            1  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 454  
2947/2947 [==============================] - 1s 264us/step


[0.08501992526514388, 0.9699129047011941]

In [1]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "Number of Layers","Dropout","Kernel Initializer","Optimizer","Log Loss","Accuracy"]

x.add_row(["LSTM",1,0.5,'Normal','RMSprop',0.42,0.90])
x.add_row(["CNN 1D",2,0.5,'Glorot Normal','Adam',0.30,0.917])
x.add_row(["CNN 1D",2,0.7,'He Normal','Adadelta',0.08,0.97])
print(x)

+--------+------------------+---------+--------------------+-----------+----------+----------+
| Model  | Number of Layers | Dropout | Kernel Initializer | Optimizer | Log Loss | Accuracy |
+--------+------------------+---------+--------------------+-----------+----------+----------+
|  LSTM  |        1         |   0.5   |       Normal       |  RMSprop  |   0.42   |   0.9    |
| CNN 1D |        2         |   0.5   |   Glorot Normal    |    Adam   |   0.3    |  0.917   |
| CNN 1D |        2         |   0.7   |     He Normal      |  Adadelta |   0.08   |   0.97   |
+--------+------------------+---------+--------------------+-----------+----------+----------+


Summary and Thought Process behind execution :
    
The initial model implemented was the LSTM model, with Normal kernel initializer,single layer and a dropout rate of 0.5.
With optimizer RMSprop, this model gave a log loss of 0.42 and an accuracy of 90%.

The second model implemented was CNN 1D, without any hyper-parameter tuning. This gave an improved score of accuracy 91.7% and
log loss of 0.3

After performing hyper-parameter tuning on the CNN 1D model, with the final parameters, the accuracy boosted upto 97% and the log loss
reduced considerably to 0.08.


    